In [1]:
DF_PATH       = "../data/processed/2_cleaned_data.pkl"

ROLE_COLS      = ['DevType']
TECH_COLS      = ['LanguageWorkedWith',    'DatabaseWorkedWith',    'WebframeWorkedWith',    'MiscTechWorkedWith']

EXPERIMENT_NAME = "stackoverflow_single_model"
LOG_PATH = "../models/temp/baseline/"
LOG_DATA_PKL    =  "data.pkl"
LOG_MODEL_PKL   =  "model.pkl"
LOG_METRICS_PKL =  "metrics.pkl"

In [2]:
# Load packages
import pandas as pd 
import numpy as np
import logging
import pickle
import random
import plotly 
import os

import mlflow
from mlflow.tracking import MlflowClient

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.pipeline import make_pipeline, FeatureUnion
from sklearn.feature_selection import VarianceThreshold
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn import metrics
from sklearn.metrics import auc, accuracy_score, confusion_matrix, f1_score, precision_score, recall_score

from sklearn.decomposition import PCA, KernelPCA

from sklearn import tree
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

from matplotlib import pyplot as plt

In [3]:
# Read Data 
df = pd.read_pickle(DF_PATH)

### Functions

In [4]:
def calculate_quality(ground_truth, prediction, metric_function, sort_values=False):
    quality_scores = {}
    for col in predictions.columns:
        role_pred  = predictions[col].copy()
        role_truth = ground_truth[col].copy()
        quality_scores[col] = round(metric_function(role_truth, role_pred) * 100, 2)
        
    quality_scores = pd.Series(quality_scores.values(), index=quality_scores.keys())
    if sort_values:
        quality_scores = quality_scores.sort_values()
    
    return quality_scores

## Balance classes

In [5]:
# Check the total samples of roles
roles_df = df["DevType"].copy()
role_sum = df["DevType"].sum(axis=0)
role_sum

Academic researcher                               581
Data or business analyst                          669
Data scientist or machine learning specialist     799
Database administrator                            296
DevOps specialist                                 677
Developer, QA or test                             493
Developer, back-end                              5503
Developer, desktop or enterprise applications    1671
Developer, embedded applications or devices       795
Developer, front-end                             2890
Developer, full-stack                            5578
Developer, game or graphics                       342
Developer, mobile                                1859
Engineer, data                                    483
Scientist                                         292
System administrator                              440
dtype: int64

In [6]:
# Resample roles
samples_per_class = 500
resampled_roles = []

for role_col in roles_df.columns:
    sub_df = roles_df.loc[roles_df[role_col] == 1].copy()
    
    if len(sub_df) < samples_per_class:
        # Upsample
        sub_df = sub_df.sample(samples_per_class, replace=True, random_state=0)
    else:
        # Downsample
        sub_df = sub_df.sample(samples_per_class, random_state=0) 
    
    resampled_roles.append(sub_df)

In [7]:
# Construct dfs
roles_df  = pd.concat(resampled_roles)
df = df.loc[roles_df.index]

In [8]:
roles_df.sum(axis=0)

Academic researcher                               797
Data or business analyst                          767
Data scientist or machine learning specialist     871
Database administrator                            588
DevOps specialist                                 705
Developer, QA or test                             590
Developer, back-end                              1770
Developer, desktop or enterprise applications     892
Developer, embedded applications or devices       666
Developer, front-end                              893
Developer, full-stack                            1431
Developer, game or graphics                       560
Developer, mobile                                 800
Engineer, data                                    631
Scientist                                         642
System administrator                              647
dtype: int64

## Prep

In [9]:
X_train, X_test, Y_train, Y_test = train_test_split(df.drop("DevType", axis=1), 
                                                    df["DevType"], 
                                                    random_state=0)

C:\Users\Omnia Hosny\anaconda3\lib\site-packages\pandas\core\generic.py:4153: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [10]:
X_train.shape

(6000, 91)

## Train models

## Initialize MLflow

In [11]:
# Initialize client and experiment

client = MlflowClient()
mlflow.set_experiment(EXPERIMENT_NAME)
exp = client.get_experiment_by_name(EXPERIMENT_NAME)

## 1. Logistic regression

In [12]:
clf = make_pipeline(StandardScaler(), 
                    MultiOutputClassifier(LogisticRegression()))

clf.fit(X_train, Y_train)
predictions =  pd.DataFrame(clf.predict(X_train), columns=Y_train.columns)

In [13]:
# Evaluate on training set
predictions =  pd.DataFrame(clf.predict(X_train), columns=Y_train.columns)
train_scores = {score.__name__: calculate_quality(Y_train, predictions, score) 
                for score in [accuracy_score, precision_score, recall_score, f1_score]}
train_scores = pd.concat(train_scores,axis=1)

In [14]:
# Evaluate on test set
predictions =  pd.DataFrame(clf.predict(X_test), columns=Y_test.columns)
test_scores = {score.__name__: calculate_quality(Y_test, predictions, score) 
                for score in [accuracy_score, precision_score, recall_score, f1_score]}
test_scores = pd.concat(test_scores,axis=1)
mean_test_scores = test_scores.mean()

In [15]:
print(mean_test_scores)
test_scores.sort_values("f1_score")

accuracy_score     91.190625
precision_score    65.601250
recall_score       28.439375
f1_score           37.190625
dtype: float64


,accuracy_score,precision_score,recall_score,f1_score
"Developer, QA or test",92.10,100.00,2.47,4.82
Database administrator,92.60,39.13,6.29,10.84
"Developer, desktop or enterprise applications",89.35,55.56,11.47,19.01
Scientist,92.65,52.78,12.75,20.54
Academic researcher,90.25,62.00,14.98,24.12
System administrator,91.85,56.00,16.57,25.57
"Engineer, data",92.85,69.39,20.99,32.23
"Developer, embedded applications or devices",92.45,57.47,30.49,39.84
"Developer, full-stack",84.50,63.13,31.65,42.16
"Developer, back-end",80.55,61.45,34.30,44.03


## Log

In [16]:
# Data details
data_details = {"data_path": DF_PATH,
                "training_indices": X_train.index.tolist(),
                "test_indices":     X_test.index.tolist(), 
                "features_names":   X_train.columns.droplevel(0).tolist(),
                "targets_names":    Y_train.columns.tolist()}

with open(os.path.join(LOG_PATH, LOG_DATA_PKL), "wb") as output_file:
    pickle.dump(data_details, output_file)

In [17]:
# Model
model = {"model_description": "Baseline model: Logistic Regression ",
         "model_details": str(clf),
         "model_object": clf} 

with open(os.path.join(LOG_PATH, LOG_MODEL_PKL), "wb") as output_file:
    pickle.dump(model, output_file)

In [18]:
# Preformance details
classes_metrics = {"train_scores": train_scores, 
                   "test_scores":  test_scores}

with open(os.path.join(LOG_PATH, LOG_METRICS_PKL), "wb") as output_file:
    pickle.dump(classes_metrics, output_file)

In [19]:
# Start a new run and track 
with mlflow.start_run(experiment_id=exp.experiment_id, run_name=model["model_description"]):
    # Log pickles 
    mlflow.log_artifacts(LOG_PATH)
    
    # Track metrics 
    for metric, score in mean_test_scores.items():
        mlflow.log_metric(metric, score) 